In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import model_banana_2D
import numpy as np

from lsmcmc import algorithms, logging, output, sampling, storage

In [ ]:
x_value = np.linspace(-1, 1, 1000)
y_value = np.linspace(-0.5, 1, 1000)
x_value, y_value = np.meshgrid(x_value, y_value)
density = model_banana_2D.evaluate_density(x_value, y_value)
model_banana_2D.plot_density(x_value, y_value, density)

In [ ]:
acceptance_rate_output = output.MCMCOutput(
    output.AcceptanceQoI(),
    output.RunningMeanStatistic(),
    f"{'Accept Rate':<12}",
    "<+12.3e",
    log=True,
)
c0_output = output.MCMCOutput(
    output.ComponentQoI(0),
    output.IdentityStatistic(),
    f"{'Component 0':<12}",
    "<+12.3e",
    log=True
)
running_mean_c0_output = output.MCMCOutput(
    output.ComponentQoI(0),
    output.RunningMeanStatistic(),
    f"{'Run_mean_C0':<12}",
    "<+12.3e",
    log=True,
)
batch_mean_c0_output = output.MCMCOutput(
    output.ComponentQoI(0),
    output.BatchMeanStatistic(1000),
    f"{'Batch_mean_C0':<14}",
    "<+14.3e",
    log=True,
)
outputs = (acceptance_rate_output, c0_output, running_mean_c0_output, batch_mean_c0_output)

In [ ]:
logger_settings = logging.LoggerSettings(
    do_printing=True,
    logfile_path=Path("logfile.log"),
    write_mode="w",
)
algorithm_settings = algorithms.AlgorithmSettings(
    step_width=0.4,
    proposal_rng=np.random.default_rng(seed=0),
    accept_reject_rng=np.random.default_rng(seed=1),
)
sampler_settings = sampling.SamplerRunSettings(
    num_samples=10000,
    initial_state=np.array([-0.5, 0.2]),
    print_interval=1000,
    store_interval=1,
)

sample_storage = storage.NumpyStorage()
posterior_model = model_banana_2D.BananaModel()
logger = logging.MCMCLogger(logger_settings)
algorithm = algorithms.pCNAlgorithm(algorithm_settings, posterior_model)
sampler = sampling.Sampler(algorithm, sample_storage, outputs, logger)
samples, outputs = sampler.run(sampler_settings)

In [ ]:
_, ax = plt.subplots(figsize=(4, 4), layout="constrained")
ax.scatter(samples.values[0, :], samples.values[1, :], alpha=0.05)